<a href="https://colab.research.google.com/github/AFalabella0/SpeechBrain/blob/master/Copia_di_prova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install speechbrain

     |████████████████████████████████| 358kB 7.5MB/s 
     |████████████████████████████████| 1.9MB 13.4MB/s 
     |████████████████████████████████| 1.2MB 49.9MB/s 
     |████████████████████████████████| 645kB 48.4MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 552kB 49.4MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!sudo apt-get install p7zip-full
!7za x /content/drive/MyDrive/Dati/LibriSpeech.7z

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Dati/                                      1 file, 685409744 bytes (654 MiB)

Extracting archive: /content/drive/MyDrive/Dati/LibriSpeech.7z
--
Path = /content/drive/MyDrive/Dati/LibriSpeech.7z
Type = 7z
Physical Size = 685409744
Headers Size = 44523
Method = LZMA2:24
Solid = +
Blocks = 1

  0%

In [ ]:
import speechbrain as sb
import torch
import os
import pandas as pd
import librosa

In [ ]:
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
currentPath = os.getcwd()
path = currentPath + "/LibriSpeech-dev/dev-clean"
df = pd.DataFrame()
for speaker in os.listdir(path):
    path1 = path + "/" + speaker
    for book in os.listdir(path1):
        path2 = path1 + "/" + book
        for name in  os.listdir(path2):
            if "trans" not in name:
                path3 = path2 + "/" + name;
                #print(path3)
                sId = name.split("-")[0]
                wave, sr = librosa.load(path3, duration = 1)
                
                df = df.append([[sId, wave]], ignore_index = True)
df.columns = ["speaker", "wave"]

In [ ]:
df.head

<bound method NDFrame.head of      speaker                                               wave
0       6295  [0.00025780825, -0.000261313, -0.0010263863, -...
1       6295  [8.710243e-05, -4.7979058e-05, -0.00019242437,...
2       6295  [-0.00091540796, -0.00085670827, -0.0005096812...
3       6295  [-0.00071276503, -0.00084933266, -0.0007747779...
4       6295  [0.0013052851, 0.0014992316, 0.0013062187, 0.0...
...      ...                                                ...
2698    7850  [-0.000115891926, -0.00016917606, -0.000228682...
2699    7850  [-0.00012614857, -0.00026312433, -0.0003505998...
2700    7850  [0.00026585275, 8.803779e-05, -0.00013858856, ...
2701    7850  [-5.7916815e-05, 7.703993e-05, 0.00024347584, ...
2702    7850  [6.122556e-05, 7.189608e-05, 3.8974264e-05, 3....

[2703 rows x 2 columns]>

In [ ]:
class SimpleBrain(sb.Brain):
  def compute_forward(self, batch, stage):
    return self.modules.model(batch["input"])

    
  def compute_objectives(self, predictions, batch, stage):
    return torch.nn.functional.l1_loss(predictions, batch["target"])

model = torch.nn.Linear(in_features=10, out_features=10)
brain = SimpleBrain({"model": model}, opt_class=lambda x: torch.optim.SGD(x, 0.1))
data = [{"input": torch.rand(10, 10), "target": torch.rand(10, 10)}]
brain.fit(range(10), data)